In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
import csv
import os
import ast
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
from keras.optimizers import SGD, Adam

In [25]:
sample = Image.open('/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Train/angry/Aishwarya_Rai67.png')
img_height, img_width = sample.size
print(img_height, img_width)

182 182


In [76]:
num_classes = 4
width, height = 182, 182
num_epochs = 20
batch_size = 32
num_features = 64

In [ ]:
model = Sequential()

#module 1
model.add(Conv2D(2*num_features, kernel_size=(3, 3), input_shape=(width, height, 1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(2*num_features, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#module 2
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#module 3
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

#flatten
model.add(Flatten())

#dense 1
model.add(Dense(2*2*2*num_features))
model.add(BatchNormalization())
model.add(Activation('relu'))

#dense 2
model.add(Dense(2*2*num_features))
model.add(BatchNormalization())
model.add(Activation('relu'))

#dense 3
model.add(Dense(2*num_features))
model.add(BatchNormalization())
model.add(Activation('relu'))

#output layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), 
              metrics=['accuracy'])

model.summary()

In [46]:
emotions = ['angry', 'happy', 'neutral', 'sad']
emotion_dict = {"angry": 0, "happy": 1, "neutral": 2, "sad": 3}

In [6]:
# Preparing the training dataset

train_dir_path = '/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Train'

train_data = []
for emotion in emotions:
  img_files = [f for f in os.listdir(f"{train_dir_path}/{emotion}") if f.endswith('.png')]

  for img_file in img_files:
    img = Image.open(os.path.join(f"{train_dir_path}/{emotion}", img_file))

    grayscale_image = img.convert("L")

    resized_image = grayscale_image.thumbnail((182, 182))

    pixels = list(grayscale_image.getdata())

    train_data.append((pixels, emotion))


In [ ]:
train_data[0]

In [7]:
# Preparing the testing dataset

test_dir_path = '/content/drive/MyDrive/BrainDead/dataset/challenge2/brain_dead_emotion_detection/Test'

test_data = []
for emotion in emotions:
  img_files = [f for f in os.listdir(f"{test_dir_path}/{emotion}") if f.endswith('.png')]

  for img_file in img_files:
    img = Image.open(os.path.join(f"{test_dir_path}/{emotion}", img_file))

    grayscale_image = img.convert("L")

    resized_image = grayscale_image.thumbnail((182, 182))

    pixels = list(grayscale_image.getdata())

    test_data.append((pixels, emotion))

In [ ]:
train_data[0][0]

In [8]:
df = pd.DataFrame(train_data, columns=['pixels', 'emotions'])
df = df.sample(frac = 1)

In [ ]:
df['pixels'][0]

In [9]:
# Preprocessing the images and labels
X_train = []
y_train = []

for i in range(len(df)):
  pixels = df['pixels'][i]
  temp = []
  for pixel in pixels:
    temp.append(pixel)
  
  pixels = np.array(temp, dtype='float32')
  pixels /= 255. # normalize
  X_train.append(pixels)
  y_train.append(df['emotions'][i])


In [11]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], 182,182,1)

In [12]:
num_classes = len(np.unique(y_train))
for i in range(len(y_train)):
  y_train[i] = emotion_dict[y_train[i]]
y_train[0]

0

In [13]:
train_emotions = to_categorical(y_train, num_classes)

In [ ]:
del(train_data)

In [78]:
model.fit(X_train, train_emotions, validation_split=0.2, epochs=num_epochs, batch_size=32, verbose=True)

Epoch 1/20
46/46 [==============================] - 44s 786ms/step - loss: 1.1351 - accuracy: 0.5110 - val_loss: 1.8808 - val_accuracy: 0.0000e+00
Epoch 2/20
46/46 [==============================] - 30s 649ms/step - loss: 0.7446 - accuracy: 0.6858 - val_loss: 3.9302 - val_accuracy: 0.0000e+00
Epoch 3/20
46/46 [==============================] - 30s 656ms/step - loss: 0.5778 - accuracy: 0.7597 - val_loss: 4.7908 - val_accuracy: 0.0000e+00
Epoch 4/20
46/46 [==============================] - 30s 651ms/step - loss: 0.3843 - accuracy: 0.8481 - val_loss: 4.8906 - val_accuracy: 0.0000e+00
Epoch 5/20
46/46 [==============================] - 30s 658ms/step - loss: 0.2516 - accuracy: 0.9109 - val_loss: 4.6570 - val_accuracy: 0.0000e+00
Epoch 6/20
46/46 [==============================] - 30s 648ms/step - loss: 0.2094 - accuracy: 0.9337 - val_loss: 7.2115 - val_accuracy: 0.0000e+00
Epoch 7/20
46/46 [==============================] - 30s 649ms/step - loss: 0.1678 - accuracy: 0.9413 - val_loss: 3.406

In [79]:
df = pd.DataFrame(test_data, columns=['pixels', 'emotions'])
df = df.sample(frac = 1)

In [80]:
# Preprocessing the images and labels
X_test = []
y_test = []

for i in range(len(df)):
  pixels = df['pixels'][i]
  temp = []
  for pixel in pixels:
    temp.append(pixel)
  
  pixels = np.array(temp, dtype='float32')
  pixels /= 255. # normalize
  X_test.append(pixels)
  y_test.append(df['emotions'][i])

In [81]:
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], 182,182,1)

In [82]:
for i in range(len(y_test)):
  y_test[i] = emotion_dict[y_test[i]]

In [83]:
y_test[0]

0

In [84]:
test_emotions = to_categorical(y_test, num_classes)

In [85]:
model.metrics_names

['loss', 'accuracy']

In [ ]:
del(test_data)

In [86]:
scores = model.evaluate(X_test, test_emotions, verbose=1)

12/12 [==============================] - 4s 336ms/step - loss: 1.9283 - accuracy: 0.5935


In [39]:
scores

[2.5202136039733887, 0.4878048896789551]